# Descarga de resultados de elecciones generales disponibles en el Atlas Electoral del OEP

El Órgano Electoral mantiene un [atlas con datos de resultados electorales](https://atlaselectoral.oep.org.bo/). En este notebook descargo todos los datos disponibles para elecciones generales.

In [104]:
import requests
import pandas as pd
import os
from IPython.display import Markdown, display

Creamos una lista de los procesos electorales disponibles en el Atlas Electoral.

In [12]:
def listar_procesos():
    def parse_categoria(categoria):
        def parse_level(level, sublevel_key):
            return [
                {k: level[k] for k in level.keys() if k != sublevel_key},
                level[sublevel_key],
            ]

        data = []
        data_categoria, procesos = parse_level(categoria, "proceso_electorals")
        for proceso in procesos:
            data_proceso, subprocesos = parse_level(proceso, "sub_procesos")
            for subproceso in subprocesos:
                data.append({**data_categoria, **data_proceso, **subproceso})
        return data

    response = requests.get(
        "https://atlaselectoral.oep.org.bo/api/proceso-electoral-menu"
    )
    resultado = []
    for categoria in response.json():
        resultado.extend(parse_categoria(categoria))
    return pd.DataFrame(resultado)

In [13]:
procesos = listar_procesos()

In [17]:
Markdown(
    f"Existen {len(procesos.id_agrupacion_proceso.unique())} categorías con {len(procesos.id_proceso_electoral.unique())} procesos y {len(procesos.id_sub_proceso.unique())} subprocesos electorales."
)

Existen 7 categorías con 48 procesos y 130 subprocesos electorales.

Éstos son los subprocesos en la categoría "_Elecciones Generales_":

In [80]:
procesos[procesos.nom_agrupacion_proceso == "Elecciones Generales"][
    ["nombre_proceso_electoral", "nom_sub_proceso", "id_sub_proceso"]
]

,nombre_proceso_electoral,nom_sub_proceso,id_sub_proceso
0,Elecciones Generales 1979,Elecciones Generales 1979,52
1,Elecciones Generales 1980,Elecciones Generales 1980,53
2,Elecciones Generales 1985,Elecciones Generales 1985,54
3,Elecciones Generales 1989,Elecciones Generales 1989,55
4,Elecciones Generales 1993,Elecciones Generales 1993,56
5,Elecciones Generales 1997,Elecciones Generales 1997,57
6,Elecciones Generales 1997,Elección de Diputados Uninominales 1997,60
7,Elecciones Generales 2002,Elecciones Generales 2002,58
8,Elecciones Generales 2002,Elección de Diputados Uninominales 2002,79
9,Elecciones Generales 2005,Elecciones Generales 2005,19


Intentamos descargar resultados para estos procesos.

In [116]:
def descargar_resultados(subproceso):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:141.0) Gecko/20100101 Firefox/141.0",
    }

    columnas_admin = {
        "Codigo PAIS": "pais",
        "Nombre PAIS": "pais_nombre",
        "Codigo DEPARTAMENTO": "departamento",
        "Nombre DEPARTAMENTO": "departamento_nombre",
        "Codigo PROVINCIA": "provincia",
        "Nombre PROVINCIA": "provincia_nombre",
        "Codigo MUNICIPIO": "municipio",
        "Nombre MUNICIPIO": "municipio_nombre",
        "Codigo CIRCUNSCRIPCION": "circunscripcion",
        "Nombre CIRCUNSCRIPCION": "circunscripcion_nombre",
        "Codigo RECINTO": "recinto",
        "Nombre RECINTO": "recinto_nombre",
        "Codigo MESA": "mesa",
        "Nombre MESA": "mesa_nombre",
    }
    directorio = subproceso["nombre_proceso_electoral"].split()[-1]
    filename = subproceso["nom_sub_proceso"].lower().replace(" ", "_")
    url = f"https://atlaselectoral.oep.org.bo/descarga/{subproceso['id_sub_proceso']}/votos_totales.csv"
    os.makedirs(directorio, exist_ok=True)

    try:
        df = pd.read_csv(url, sep="|", storage_options=headers)
        admin = [col for col in df.columns if col in columnas_admin.keys()]
        stacked = df.set_index(admin).stack().reset_index().copy()
        stacked.columns = [columnas_admin[col] for col in admin] + ["opcion", "conteo"]
        stacked.to_parquet(f"{directorio}/{filename}.parquet")
        display(
            Markdown(
                f"**{subproceso['nom_sub_proceso']}**: desagregado en {', '.join(['`' + col.split()[-1].lower() + '`' for col in admin if 'Codigo' in col])}"
            )
        )
    except Exception as e:
        display(Markdown(f"{subproceso['nom_sub_proceso']}: ERROR {e}"))

In [118]:
for i, subproceso in procesos[
    procesos.nom_agrupacion_proceso == "Elecciones Generales"
].iterrows():
    descargar_resultados(subproceso)

**Elecciones Generales 1979**: desagregado en `pais`, `departamento`

**Elecciones Generales 1980**: desagregado en `pais`, `departamento`

**Elecciones Generales 1985**: desagregado en `pais`, `departamento`, `provincia`

**Elecciones Generales 1989**: desagregado en `pais`, `departamento`, `provincia`

**Elecciones Generales 1993**: desagregado en `pais`, `departamento`, `provincia`

...

**Elección de Diputados en Circunscripciones Especiales 2014**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

**ELECCIONES GENERALES 2020**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

**ELECCIONES GENERALES 2020 EXTERIOR**: desagregado en `pais`, `recinto`, `mesa`

**ELECCIÓN DE DIPUTADOS UNINOMINALES 2020**: desagregado en `pais`, `circunscripcion`, `recinto`, `mesa`

**ELECCIÓN DE DIPUTADOS EN CIRCUNSCRIPCIONES ESPECIALES 2020**: desagregado en `pais`, `departamento`, `provincia`, `municipio`, `recinto`, `mesa`

Podemos descargar resultados para todos los procesos excepto las elecciones de diputados uninominales de 2005, para las que [no existen](https://atlaselectoral.oep.org.bo/descarga/80/votos_totales.csv) datos en el servidor. Además tenemos datos a resolución de mesa desde la elección general de 2002. Con una variación mínima es posible descargar datos para otros procesos electorales, que incluyen elecciones departamentales, municipales, judiciales, referendos, elecciones constituyentes y elecciones primarias.